<a href="https://colab.research.google.com/github/sahandv/science_science/blob/master/FastText_embedding-Copy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FASTTEXT EMBEDDING


## Init

Local OR Colab?

In [7]:
# datapath = '/mnt/16A4A9BCA4A99EAD/GoogleDrive/Data/' # Local
datapath = 'drive/My Drive/Data/' # Remote

### Clone Project Git Repo

In [2]:
!rm -rf 'science_science'
username = "sahandv"#@param {type:"string"}
# password = ""#@param {type:"string"} 

!git clone https://github.com/$username/science_science.git
!ls

Cloning into 'science_science'...
remote: Enumerating objects: 297, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (264/264), done.
remote: Total 1299 (delta 165), reused 151 (delta 32), pack-reused 1002
Receiving objects: 100% (1299/1299), 102.71 MiB | 18.18 MiB/s, done.
Resolving deltas: 100% (760/760), done.
sample_data  science_science


### Mount Google Drive


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Check files!
!ls 'drive/My Drive/Data-Permenant/FastText-crawl-300d-2M-subword'

### Install requirements

In [6]:
# !pip install fasttext
!pip install -r 'science_science/requirements.txt'
!pip install gensim==3.8.1

### Import Libs

In [8]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import gc
import json
import re

# import fasttext
import gensim
from gensim.models import FastText as fasttext_gensim
from gensim.test.utils import get_tmpfile

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# from sciosci.assets import keyword_assets as kw
# from sciosci.assets import generic_assets as sci
# from sciosci.assets import advanced_assets as aa

from science_science.sciosci.assets import keyword_assets as kw
from science_science.sciosci.assets import generic_assets as sci
from science_science.sciosci.assets import advanced_assets as aa

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
stops = ['a','an','we','result','however','yet','since','previously','although','propose','proposed','e_g','method',
         'published_elsevier','b','v','problem','paper','approach','within','with','by','via','way','t','case','issue','level','area','system',
         'work','discussed','seen','put','usually','take','make','author','versus','enables','result','research','design','based']
punkts = [' ','','(',')','[',']','{','}','.',',','!','?','<','>','-','_',':',';','\\','/','|','&','%',"'s","`s",'#','$','@']

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = list(set(stopwords.words("english")))+stops+punkts
np.random.seed(50)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Get embeddings from a pre-trained model


### Load Corpus

In [ ]:
period = '2017-2018'
percentile = 97

#### Option A - Load corpus

In [ ]:
directory = datapath+'Corpus/Unsupervised Training/Sentences/'
file_name = 'sentences cora_wos training'#corpus abstract-title - with n-grams'
corpus = pd.read_csv(directory+file_name,names=['abstracts'])
corpus_tokens = [item.lower() for sublist in corpus['abstracts'].values.tolist() for item in sublist.split()]
gc.collect()

42

In [ ]:
corpus

,abstracts
0,lighting system disclosed comprising plurality...
1,photographic device eliminates photometric err...
2,print demand camera system camera unit incorpo...
3,simplified camera mechanism method allowing op...
4,beam splitter digital camera split light passe...
...,...
19705,display engine video graphic system includes p...
19706,facilitating interaction may enabled communica...
19707,present invention generally directed system me...
19708,disclosed system producing image including app...


#### Option B - Load keywords

In [ ]:
directory = datapath+'LDA/'
file_name = period+' top_90-percentile_keywords_terms.csv'
corpus = pd.read_csv(directory+file_name)
corpus = corpus.fillna('this_is_null')
corpus_tokens = []
for idx,row in tqdm(corpus.iterrows(),total=corpus.shape[0]):
    for token in row.values.tolist():
        if token != 'this_is_null': 
            corpus_tokens.append(token) 
del corpus
print("\nNumber of unique tokens:",len(corpus_tokens))

#### Option C - Load author keywords

In [ ]:
directory = datapath+'Author keywords - 29 Oct 2019/'
file_name = period+' keyword frequency'
corpus = pd.read_csv(directory+file_name,names=['keyword','frequency'])
corpus = corpus.fillna('this_is_null')
threshold = np.percentile(corpus['frequency'].values,percentile)
corpus = corpus[corpus['frequency']>threshold]

corpus_tokens = []
for idx,row in tqdm(corpus.iterrows(),total=corpus.shape[0]):
    if row['keyword'] != 'this_is_null': 
        corpus_tokens.append(row['keyword']) 
print("\nNumber of unique tokens:",len(corpus_tokens))

## Facebook Model

#### Load model

In [ ]:
fb_model_address = datapath+'/FastText-crawl-300d-2M-subword/crawl-300d-2M-subword.bin'

In [ ]:
model = fasttext.load_model(fb_model_address)

#### Get embeddings

*   Save to dictionary and json
*   This takes much less space on disk


##### No n-gram handle

In [ ]:
# Save in a dict
output_dict = {}
comment_embedding = ''
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    output_dict[token] = str(model.get_word_vector(token))

##### Manual n-gram handle

In [ ]:
# Save in a dict
comment_embedding = 'average_manual '
output_dict = {}
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    token_split = token.split(' ')
    if len(token_split) > 0:
        tmp_vector_grams = []
        for item in token_split:
            tmp_vector_grams.append(model.get_word_vector(item))
        output_dict[token] = str(list(np.array(tmp_vector_grams).mean(axis=0)))
    else:
        output_dict[token] = str(model.get_word_vector(item))

##### Save to disk

In [ ]:
# Save embeddings to disk
with open(directory+'vectors/100D/FastText vector '+comment_embedding+period+'.json', 'w') as json_file:
    json.dump(output_dict, json_file)

#### Get embeddings (alternative) : 

*   save to a list instead of a dicktionary and csv
*   Will have many redundant words in it and will take lots of disk space




In [ ]:
# Save in a list
batches = 1000
batch_size = len(corpus_tokens)/batches

for step in tqdm(range(batches),total=batches):
    batch_tokens = corpus_tokens[int(step*batch_size):int((step+1)*batch_size)]
    corpus_vectors = [model.get_word_vector(x) for x in batch_tokens]
    corpus_vectors = pd.DataFrame(corpus_vectors)
    corpus_vectors['tokens'] = batch_tokens

    # Save embeddings to disk
    with open(directory+'vector '+period,'a') as f:
        corpus_vectors.to_csv(f,index=False,header=False)

## Gensim Model

#### Load model

Load gensim model

In [ ]:
gensim_model_address = datapath+'FastText Models/50D w1/fasttext-scopus-2.2-million_docs-gensim 50D-w1.model'
model = fasttext_gensim.load(gensim_model_address)

In [ ]:
gensim_model_address = datapath+"FastText Models/100D Oct2020/deflemm/patent_wos corpus.model"
model = fasttext_gensim.load(gensim_model_address)

In [ ]:
gensim_model_address = 'drive/My Drive/Data-Permenant/FastText-crawl-300d-2M-subword/crawl-300d-2M-subword.bin'
model = gensim.models.fasttext.load_facebook_model(gensim_model_address)

Simple Tests

In [ ]:
print('intelligence' in model.wv.vocab)
print(model.similarity("anns", "ann"))
print(model.most_similar(['eye','vision','processing'], ['computer']))
print(model.wmdistance(['stop', 'word', 'removed', 'tokens', 'of', 'sentence 1'], ['stop word removed tokens of sentence 2']))

In [ ]:
# get distance of two words
from scipy import spatial,sparse,sign
vec_a = model.wv['']
vec_b = model.wv['fpga']
distance_tmp = spatial.distance.cosine(vec_a, vec_b)
similarity_tmp = 1 - distance_tmp
similarity_tmp

In [ ]:
model.wv['artificial intelligence']

In [ ]:
(model.wv['artificial']+model.wv['intelligence'])/2

In [ ]:
model.wv['artificial_intelligence']

#### Compare vectors to ACM categories

In [ ]:
AI_categories = [
              'Artificial Intelligence Applications - Expert Systems',
              'Automatic Programming',
              'Deduction - Theorem Proving',
              'Knowledge Representation Formalisms - Knowledge Representation Methods',
              'Programming Languages - Software',
              'Learning',
              'Natural Language Processing',
              'Problem Solving - Control Methods - Search',
              'Robotics',
              'Vision - Scene Understanding',
              'Distributed Artificial Intelligence',
              'ARTIFICIAL INTELLIGENCE'
]

In [ ]:
categories = [
              'Natural language processing - Information extraction - Machine translation - Discourse, dialogue  pragmatics - Natural language generation - Speech recognition - Lexical semantics - Phonology / morphology',
              'Knowledge representation reasoning - Description logics - Semantic networks Nonmonotonic default reasoning  belief revision - Probabilistic reasoning - Vagueness fuzzy logic - Causal reasoning  diagnostics - Temporal reasoning - Cognitive robotics - Ontology engineering - Logic programming answer set programming - Spatial  physical reasoning - Reasoning about belief  knowledge',
              'Planning  scheduling - Planning for deterministic actions - Planning under uncertainty - Multi-agent planning - Planning  abstraction  generalization - Robotic planning - Evolutionary robotics',
              'Search methodologies - Heuristic function construction - Discrete space search - Continuous space search - Randomized search - Game tree search - Abstraction  micro-operators - Search with partial observations - ',
              'Control methods - Robotic planning - Evolutionary robotics - Computational control theory - Motion path planning',
              'Philosophical theoretical foundations artificial intelligence - Cognitive science - Theory mind',
              'Distributed artificial intelligence - Multi agent systems - Intelligent agents - Mobile agents - Cooperation  coordination',
              'Computer vision - Biometrics - Scene understanding - Activity recognition  understanding - Video summarization - Visual content based indexing  retrieval - Visual inspection - Vision for robotics - Scene anomaly detection - Image  video acquisition - Camera calibration - Epipolar geometry - Computational photography - Hyperspectral imaging - Motion capture - 3D imaging - Active vision - Image representations - Shape representations - Appearance  texture - Hierarchical representations - Computer vision problems - Interest point  salient region detections - Image segmentation  - Video segmentation - Shape inference - Object detection - Object recognition - Object identification - Tracking - Reconstruction - Matching',
              
              'Learning paradigms - Supervised learning - Ranking - Learning to rank - Supervised learning  classification - Supervised learning  regression - Structured outputs - Cost sensitive learning - Unsupervised learning - Cluster analysis - Anomaly detection - Mixture modeling - Topic modeling - Source separation - Motif discovery - Dimensionality reduction  manifold learning - Reinforcement learning - Sequential decision making - Inverse reinforcement learning - Apprenticeship learning - Multi-agent reinforcement learning - Adversarial learning - Multi-task learning - Transfer learning - Lifelong machine learning - Learning under covariate shift',
              'Learning settings - Batch learning - Online learning settings - Learning from demonstrations - Learning from critiques - Learning from implicit feedback - Active learning settings - Semi supervised learning settings',
              'Machine learning approaches - Classification  regression trees - Kernel methods - Support vector machines - Gaussian processes - Neural networks - Logical  relational learning - Inductive logic learning - Statistical relational learning - Learning in probabilistic graphical models - Maximum likelihood modeling - Maximum entropy modeling - Maximum a posteriori modeling - Mixture models - Latent variable models - Bayesian network models - Learning linear models - Perceptron algorithm - Factorization methods - Non-negative matrix factorization - Factor analysis - Principal component analysis - Canonical correlation analysis - Latent Dirichlet allocation - Rule learning - Instance-based learning - Markov decision processes -  Markov decision processes - Stochastic games - Learning latent representations - Deep belief networks - Bio inspired approaches - Artificial life - Evolvable hardware - Genetic algorithms - Genetic programming - Evolutionary robotics - Generative  developmental approaches',
              'Machine learning algorithms - Dynamic programming Markov decision processes - Value iteration - Q learning - Policy iteration - Temporal difference learning - Approximate dynamic programming methods - Ensemble methods - Boosting - Bagging - Spectral methods - Feature selection - Regularization',
]

In [ ]:
AI_vectors = []
labels = []
for item in categories:
    vector_tmp = []
    label = item.split('-')[0]
    for phrase in item.split('-'):
        phrase = phrase.lower().strip()
        # print(phrase)
        vector_tmp.append(model.wv[phrase])
    # print('---')
    AI_vectors.append(list(np.array(vector_tmp).mean(axis=0)))
    labels.append(label)
print(AI_vectors)

In [ ]:
pd.DataFrame(AI_vectors).to_csv(datapath+'FastText doc clusters - SIP/50D/classification/ACM_classifications_vectors')
pd.DataFrame(labels,columns=['label']).to_csv(datapath+'FastText doc clusters - SIP/50D/classification/ACM_classifications_labels')

### Get Word Embeddings

##### No n-gram handle

In [ ]:
# Save in a dict
comment_embedding = ''
output_dict = {}
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    output_dict[token] = str(model.wv[token])

##### Manual n-gram handle

In [ ]:
# Save in a dict
comment_embedding = 'average_manual '
output_dict = {}
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    token_split = token.split(' ')
    if len(token_split) > 0:
        tmp_vector_grams = []
        for item in token_split:
            tmp_vector_grams.append(model.wv[item])
        output_dict[token] = str(list(np.array(tmp_vector_grams).mean(axis=0)))
    else:
        output_dict[token] = str(model.wv[token])

##### Save to disk

In [ ]:
# Save embeddings to disk
with open(directory+'/FastText vector with n-grams '+comment_embedding+period+'.json', 'w') as json_file:
    json.dump(output_dict, json_file)

In [ ]:
print(directory)

### Get Doc Embeddings (SIF)

It is not recommended to perform this on cloud, as it is not process intesive, yet takes too long depending on the doc-count. It might take over 30 hours.

Make a probability dictionary

In [ ]:
corpus_tokens_s = pd.Series(corpus_tokens)
corpus_tokens_probabilities = (corpus_tokens_s.groupby(corpus_tokens_s).transform('count') / len(corpus_tokens_s)).values
corpus_tokens_probabilities = pd.DataFrame(corpus_tokens_probabilities)
corpus_tokens_probabilities['tokens'] = corpus_tokens_s
corpus_tokens_probabilities.columns = ['probability','token']
corpus_tokens_probabilities = corpus_tokens_probabilities.groupby('token').mean()
corpus_tokens_probabilities = corpus_tokens_probabilities.reset_index()
corpus_tokens_probabilities.columns = ['token','probability']

Get vectors

In [ ]:
vectors = []
for token in tqdm(corpus_tokens_probabilities['token'],total=corpus_tokens_probabilities.shape[0]):
    vectors.append(model.wv[token])
corpus_tokens_probabilities['vector'] = vectors

100%|██████████| 17086/17086 [00:00<00:00, 42392.45it/s]


Calculate weighted average vectors

In [ ]:
a = 1e-3
embedding_size = 100

doc_set = []
for doc in tqdm(corpus['abstracts'].values.tolist(),total=len(corpus['abstracts'].values.tolist())):
    vs = np.zeros(embedding_size)  # add all word2vec values into one vector for the sentence
    doc_length = len(doc.split())
#     print(doc.split())
    for word in doc.lower().split():
        a_value = a / (a + corpus_tokens_probabilities[corpus_tokens_probabilities['token']==word]['probability'].values.tolist()[0])  # smooth inverse frequency, SIF
        vs = np.add(vs, np.multiply(a_value, corpus_tokens_probabilities[corpus_tokens_probabilities['token']==word]['vector'].values.tolist()[0]))  # vs += sif * word_vector

    vs = np.divide(vs, doc_length)  # weighted average
    doc_set.append(vs)  # add to our existing re-calculated set of sentences


100%|██████████| 19710/19710 [50:09<00:00,  6.55it/s] 


In [ ]:
pd.DataFrame(doc_set).to_csv(datapath+'Corpus/KPRIS/embeddings/deflemm/FastText SIF patent_wos corpus',index=False)

##### Wikipedia embedding

In [ ]:
import os
import time
from stat import S_ISREG, ST_CTIME, ST_MODE

print("\nSearching for Wiki texts...\n")
dir_path = datapath+'Corpus/AI Wiki Classifications/applications/clean/'
data = (os.path.join(dir_path, fn) for fn in os.listdir(dir_path))
data = ((os.stat(path), path) for path in data)
data = ((stat[ST_CTIME], path) for stat, path in data if S_ISREG(stat[ST_MODE]))

names = []
files = []
for cdate, path in sorted(data):
    print('   - ', time.ctime(cdate), os.path.basename(path),int(os.path.getsize(path)/1000000),'MB')
    files.append(path)
    names.append(os.path.basename(path))

In [ ]:
embedding_size = 50
all = []
for file_index,file in enumerate(files):
    print('\n',file,'\n')
    corpus = pd.read_csv(file)
    corpus_tokens = [item.lower() for sublist in corpus['sentence'].values.tolist() if pd.notnull(sublist) for item in sublist.split()]
    gc.collect()
    
    corpus_tokens_s = pd.Series(corpus_tokens)
    corpus_tokens_probabilities = (corpus_tokens_s.groupby(corpus_tokens_s).transform('count') / len(corpus_tokens_s)).values
    corpus_tokens_probabilities = pd.DataFrame(corpus_tokens_probabilities)
    corpus_tokens_probabilities['tokens'] = corpus_tokens_s
    corpus_tokens_probabilities.columns = ['probability','token']
    corpus_tokens_probabilities = corpus_tokens_probabilities.groupby('token').mean()
    corpus_tokens_probabilities = corpus_tokens_probabilities.reset_index()
    corpus_tokens_probabilities.columns = ['token','probability']

    vectors = []
    for token in tqdm(corpus_tokens_probabilities['token'],total=corpus_tokens_probabilities.shape[0]):
        vectors.append(model.wv[token])
    corpus_tokens_probabilities['vector'] = vectors

    a = 1e-3

    doc_set = []
    for doc in tqdm(corpus['sentence'].values.tolist(),total=len(corpus['sentence'].values.tolist())):
        vs = np.zeros(embedding_size)  # add all word2vec values into one vector for the sentence
        if pd.notnull(doc):
            doc_length = len(doc.split())
        #     print(doc.split())
            for word in doc.lower().split():
                a_value = a / (a + corpus_tokens_probabilities[corpus_tokens_probabilities['token']==word]['probability'].values.tolist()[0])  # smooth inverse frequency, SIF
                vs = np.add(vs, np.multiply(a_value, corpus_tokens_probabilities[corpus_tokens_probabilities['token']==word]['vector'].values.tolist()[0]))  # vs += sif * word_vector

            vs = np.divide(vs, doc_length)  # weighted average
            doc_set.append(vs)  # add to our existing re-calculated set of sentences

    pd.DataFrame(doc_set).to_csv(datapath+'Corpus/AI Wiki Classifications/applications/clean/vectors/'+names[file_index],index=False)

    all.append(pd.DataFrame(doc_set).mean(axis=0))

all_df = pd.DataFrame(all)
all_df['clusters'] = names
all_df.to_csv(datapath+'Corpus/AI Wiki Classifications/applications/clean/vectors/all',index=False)

### Get Doc Embedding (averaging)

In [ ]:
doc_vectors = []
for doc in tqdm(corpus['abstracts']):
    tokens = doc.split()
    doc_vectors.append(np.array([model.wv[token] for token in tokens]).mean(axis=0))
pd.DataFrame(doc_vectors).to_csv(datapath+'Corpus/KPRIS/embeddings/FastText Avg large corpus',index=False)

100%|██████████| 19710/19710 [00:09<00:00, 2073.87it/s]


# Train on a large Scopus corpus


#### Load Corpus Sentences

In [11]:
sentence_corpus = pd.read_csv(datapath+'Corpus/Unsupervised Training/Sentences/sentences cora_wos training')

#### Preprocess and prepare corpus for FastText training

In [ ]:
sentences = []
lemmatizer=WordNetLemmatizer()
with open(datapath+'corpus/AI ALL/1900-2019 corpus sentences abstract-title further processed.csv', 'w') as f:
    for index,row in tqdm(sentence_corpus.iterrows(),total=sentence_corpus.shape[0]):
        sentence = row['sentence']
        sentence = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",sentence)
        sentence = word_tokenize(sentence)
        sentence = [word for word in sentence if not word in punkts] 
        sentence = [lemmatizer.lemmatize(word) for word in sentence]
        # sentences.append(sentence)
        f.write("%s\n" % ' '.join(sentence))

#### Save sentences to disk for future use -- Not needed anymore

In [ ]:
pd.DataFrame([' '.join(words) for words in sentences],columns=['sentences']).to_csv(
    datapath+'corpus/AI ALL/1900-2019 corpus sentences abstract-title further processed.csv',
    header=True,index=False)

#### Load pre-processed sentences

In [12]:
# sentence_corpus = pd.read_csv(datapath+'Corpus/AI ALL/1900-2019 corpus sentences abstract-title further processed.csv',delimiter=";;;")
# sentence_corpus.columns = ["sentence"]
sentences = []
sentence_corpus = sentence_corpus.fillna('')
for index,row in tqdm(sentence_corpus.iterrows(),total=sentence_corpus.shape[0]):
    sentences.append(row['sentence'].split(' '))

100%|██████████| 305226/305226 [00:32<00:00, 9360.58it/s]


In [14]:
sentence_corpus.head(10)

,article_index,sentence,year
0,0,reference c delobel r g casey,2010.0
1,0,decomposition data base theory boolean,2010.0
2,0,lemma,2010.0
3,0,therefore may iteratively repeat step generati...,2010.0
4,0,show g contains hamiltonneian path concludes p...,2010.0
5,0,p l,2010.0
6,0,hammer,2010.0
7,0,kogan,2010.0
8,0,horn function minimization knowledge compressi...,2010.0
9,0,rutcor research report rrr rutgers university ...,2010.0


In [13]:
sentences[:5]

[['reference', 'c', 'delobel', 'r', 'g', 'casey'],
 ['decomposition', 'data', 'base', 'theory', 'boolean'],
 ['lemma'],
 ['therefore',
  'may',
  'iteratively',
  'repeat',
  'step',
  'generating',
  'f',
  'etc',
  'j',
  'constructed',
  'f',
  'j',
  'path',
  'contains',
  'vertex',
  'g'],
 ['show',
  'g',
  'contains',
  'hamiltonneian',
  'path',
  'concludes',
  'proof',
  'theorem']]

### Train Fasttext - Gensim

#### Load a model to continue training

* If want to continue training, run this section

In [ ]:
model = load(gensim_model_address)

In [ ]:
model.build_vocab(sentences, update=True)
model.train(sentences, total_examples=len(sentences), epochs=model.epochs)

* Otherwise run this section

#### Train

In [ ]:
model = fasttext_gensim(min_n=3, max_n=6, size=15, window=5, min_count=1, seed = 50)
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=len(sentences), epochs=10)

In [ ]:
fname = "datapath+Models/fasttext-scopus-2.2-million_docs-gensim 15D.model"
model.save(fname)

In [ ]:
model = fasttext_gensim(min_n=3, max_n=6, size=50, window=5, min_count=1, seed = 50)
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=len(sentences), epochs=10)
fname = datapath+"Models/fasttext-scopus-2.2-million_docs-gensim 50D.model"
model.save(fname)

In [ ]:
model = fasttext_gensim(min_n=3, max_n=12, size=128, window=5, min_count=5, seed = 1)
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=len(sentences), epochs=40,threads=15)
fname = datapath+"Corpus/cora-classify/cora/models/fasttext cora_wos corpus 128d5w.model"
model.save(fname)

#### Test model

In [ ]:
similarities = model.wv.most_similar(positive=['logic','fuzzy','expert'],negative=['deep','neural','network','cnn','ann'])
most_similar = similarities[0]

In [ ]:
most_similar

In [ ]:
not_matching = model.wv.doesnt_match("human computer interface tree".split())

In [ ]:
not_matching

In [ ]:
sim_score = model.wv.similarity('computer', 'human')

In [ ]:
sim_score

In [ ]:
print(model.wv['artificial intelligence'])
print(model.wv['artificial'])
print(model.wv['intelligence'])

### Train Fasttext - Facebook

In [ ]:
sentences_joined = ' '.join(sentences)
model = fasttext.train_unsupervised(sentences_joined, "cbow", minn=2, maxn=5, dim=50, epoch=10,lr=0.05)

#### Test model

In [ ]:
model.words

In [ ]:
model.get_word_vector("the")

In [ ]:
model.get_nearest_neighbors('asparagus')

In [ ]:
model.get_analogies("intelligence", "math", "fuzzy")

#### Save model

In [ ]:
model.save_model(datapath+"fasttext-scopus_wos-merged-310k_docs-facebook.ftz")